# Demos: Lecture 14

## Demo 1: QFT from scratch

<img src="fig/qft-full.png" width="800px">

$$
R_k = \begin{pmatrix} 1 & 0 \\ 0 & e^{2\pi i/2^k} \end{pmatrix}
$$

In [1]:
import pennylane as qml
import numpy as np

In [2]:
def apply_swaps(wires=None):
    for wire_idx in range(len(wires)//2):
        qml.SWAP(wires=[wire_idx, len(wires) - wire_idx - 1])

In [3]:
def apply_controlled_rotations(target_idx=None, wires=None):
    qml.Hadamard(wires=target_idx)
    
    for idx, control_wire in enumerate(range(target_idx + 1, len(dev.wires))):
        qml.ControlledPhaseShift(
            2 * np.pi / (2 ** (idx + 2)), 
            wires=[control_wire, target_idx]
        )

In [4]:
dev = qml.device('default.qubit', wires=5)

@qml.qnode(dev)
def QFT_circuit():
    for target_wire in dev.wires:
        apply_controlled_rotations(target_wire, wires=dev.wires)
    apply_swaps(wires=dev.wires)
    return qml.state()

In [5]:
print(qml.draw(QFT_circuit)())

0: ──H─╭Rϕ(1.57)─╭Rϕ(0.79)─╭Rϕ(0.39)─╭Rϕ(0.20)──────────────────────────────────────────────
1: ────╰●────────│─────────│─────────│──────────H─╭Rϕ(1.57)─╭Rϕ(0.79)─╭Rϕ(0.39)─────────────
2: ──────────────╰●────────│─────────│────────────╰●────────│─────────│──────────H─╭Rϕ(1.57)
3: ────────────────────────╰●────────│──────────────────────╰●────────│────────────╰●───────
4: ──────────────────────────────────╰●───────────────────────────────╰●────────────────────

────────────────────────────╭SWAP───────┤  State
────────────────────────────│─────╭SWAP─┤  State
──╭Rϕ(0.79)─────────────────│─────│─────┤  State
──│──────────H─╭Rϕ(1.57)────│─────╰SWAP─┤  State
──╰●───────────╰●─────────H─╰SWAP───────┤  State


## Demo 2: QFT from template

In [6]:
@qml.qnode(dev)
def QFT_template():
    qml.QFT(wires=dev.wires)
    return qml.state()
                                  

In [7]:
print(qml.draw(QFT_template)())

0: ─╭QFT─┤  State
1: ─├QFT─┤  State
2: ─├QFT─┤  State
3: ─├QFT─┤  State
4: ─╰QFT─┤  State


In [8]:
@qml.qnode(dev)
def QFT_template():
    qml.QFT.compute_decomposition(wires=dev.wires, n_wires=len(dev.wires))
    return qml.state()         

In [9]:
print(qml.draw(QFT_template)())

0: ──H─╭Rϕ(1.57)─╭Rϕ(0.79)─╭Rϕ(0.39)─╭Rϕ(0.20)──────────────────────────────────────────────
1: ────╰●────────│─────────│─────────│──────────H─╭Rϕ(1.57)─╭Rϕ(0.79)─╭Rϕ(0.39)─────────────
2: ──────────────╰●────────│─────────│────────────╰●────────│─────────│──────────H─╭Rϕ(1.57)
3: ────────────────────────╰●────────│──────────────────────╰●────────│────────────╰●───────
4: ──────────────────────────────────╰●───────────────────────────────╰●────────────────────

────────────────────────────╭SWAP───────┤  State
────────────────────────────│─────╭SWAP─┤  State
──╭Rϕ(0.79)─────────────────│─────│─────┤  State
──│──────────H─╭Rϕ(1.57)────│─────╰SWAP─┤  State
──╰●───────────╰●─────────H─╰SWAP───────┤  State


## Demo 3: QPE for the $T$ gate

In [12]:
dev = qml.device("default.qubit", wires=4, shots=1)

@qml.qnode(dev)
def qft_for_t():
    # Prepare eigenstate |1> on last qubit
    qml.PauliX(wires=3)

    # Prepare estimation register
    for wire in range(3):
        qml.Hadamard(wires=wire)

    # Controlled rotations
    qml.ctrl(qml.PauliZ, control=0)(wires=3)
    qml.ctrl(qml.S, control=1)(wires=3)
    qml.ctrl(qml.T, control=2)(wires=3)

    # Inverse QFT
    qml.adjoint(qml.QFT)(wires=[0, 1, 2])

    return qml.sample()    

In [14]:
qft_for_t()

tensor([0, 0, 1, 1], requires_grad=True)